In [34]:
import pandas as pd
import numpy as np

In [35]:
import h5py
import os
import xarray as xr

from dask.distributed import Client
from DASStore.modules import convert
from tqdm import tqdm

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Original Sample Data

In [36]:
sample_data = h5py.File('/Users/stlp/Desktop/just1146_1147/whidbey_2022-10-20_15-47-00_UTC_002503.h5')

In [37]:
print(sample_data['Acquisition']['Raw[0]']['RawData'].shape), 
print(sample_data['Acquisition']['Raw[0]']['RawDataSampleCount'].shape),
print(sample_data['Acquisition']['Raw[0]']['RawDataTime'].shape),

(120000, 1721)
(120000,)
(120000,)


(None,)

# Random Data Created

In [38]:
# distance = np.arange(1721)
# time = np.random.random_integers(low=0, high = 500, size = (120000,))

In [39]:
# Cx = xr.DataArray(C, dims = ['time'],  name = 'RawDataTime')

In [40]:
A = np.random.rand(120000,1721)
B = np.random.rand(120000)

Ax = xr.DataArray(A, dims = ['time', 'distance'], name = 'RawData')
Bx = xr.DataArray(B, dims = ['time'],  name = 'RawDataSampleCount')

# Testing 

In [41]:
chunk_size = {'time':3000, 'distance':1721}
fn_zarr = '/Volumes/Elements_1/test.zarr'

In [ ]:
## please confirm -- is this what we want to do?
first_loop=True
for i in tqdm(range(0,10000)):
    
    # incrementally increase time coordinate
    C = np.arange(i*120000, (i+1)*120000)
    Cx = xr.DataArray(C, dims = ['time'],  name = 'RawDataTime')
    
    # creating ds
    ds = xr.Dataset({'RawData': Ax, 
                     'RawDataSampleCount': Bx, 
                     'RawDataTime': Cx})
    
    ds = ds.chunk(chunk_size)
    
    # saving data
    if first_loop:
        first_loop = False
        ds.to_zarr(fn_zarr, mode='w-')
    else:
        ds.to_zarr(fn_zarr, append_dim='time')
    

 46%|██████████████▌                 | 4559/10000 [18:55:43<31:07:10, 20.59s/it]

In [55]:
ds_zarr = xr.open_zarr('/Users/stlp/Desktop/just1146_1147/test.zarr')
ds_zarr

<xarray.Dataset>
Dimensions:             (time: 840000, distance: 1721)
Coordinates:
  * distance            (distance) int64 0 1 2 3 4 ... 1716 1717 1718 1719 1720
  * time                (time) int64 460 205 479 62 216 ... 34 231 20 178 480
Data variables:
    RawData             (time, distance) float64 dask.array<chunksize=(3000, 1721), meta=np.ndarray>
    RawDataSampleCount  (time) float64 dask.array<chunksize=(3000,), meta=np.ndarray>
    RawDataTime         (time) float64 dask.array<chunksize=(3000,), meta=np.ndarray>

In [ ]:
##### original code convert.py
#### dont run this cell

def h52zarr_xarray(h5_dir, fn_zarr, file_list, chunk):
    '''
    converts multiple h5 files to single zarr store using xarray. Manually loads the data into memory
        for single h5 and then writes/appends to zarr using xarray
    
    TODO This is pretty slow right now. Theres probably a better way to read h5 into memory.

    Parameters
    ----------
    h5_dir : string
        absolute (local) path to all h5 files to be converted. Should be folder containing
        multiple h5 files. Currently, it is assumed that each file is sorted in order
        and the append dimension is time.
    fn_zarr : string
        location of zarr store
    chunk : dict
        how to chunk zarr store (default {'time':3000, 'distance':3000})
    '''

#    file_list = os.listdir(h5_dir)
    file_list.sort()
    files = [h5_dir+file for file in file_list]
    
    first_loop=True

    for file in tqdm(files):
        if file.endswith(".h5"):
            hf = h5py.File(file)
            ds = xr.Dataset({
                'RawData':((['time', 'distance'], hf['Acquisition']['Raw[0]']['RawData'][:])),
                'RawDataSampleCount':('time', hf['Acquisition']['Raw[0]']['RawDataSampleCount'][:]),
                'RawDataTime':('time', hf['Acquisition']['Raw[0]']['RawDataTime'])})
                
            ds = ds.chunk(chunk)
                
            # create new zarr store if beginning of loop otherwize, append in time dimension
            if first_loop:
                first_loop = False
                ds.to_zarr(fn_zarr, mode='w-')
            else:
                ds.to_zarr(fn_zarr, append_dim='time')
    return 
